In [2]:
import cv2, time, os
import numpy as np
from diambra_environment.diambraGym import diambraGym

In [3]:
repo_base_path = os.path.join(os.path.abspath("/"), "home/gansandgains/DIAMBRAenvironment") # Absolute path to your DIAMBRA environment

diambraEnvKwargs = {}
diambraEnvKwargs["gameId"]          = "doapp"
diambraEnvKwargs["roms_path"]       = os.path.join(repo_base_path, "roms/") # Absolute path to roms

diambraEnvKwargs["mame_diambra_step_ratio"] = 6
diambraEnvKwargs["render"]      = True
diambraEnvKwargs["lock_fps"]    = True # Locks to 60 FPS
diambraEnvKwargs["sound"]       = diambraEnvKwargs["lock_fps"] and diambraEnvKwargs["render"]

# 1P
diambraEnvKwargs["player"] = "Random"
# 2P
#diambraEnvKwargs["player"] = "P1P2"

# Game specific
diambraEnvKwargs["difficulty"] = 3
diambraEnvKwargs["characters"]  = [["Random", "Random"], ["Random", "Random"]]
diambraEnvKwargs["charOutfits"] = [2, 2]

In [4]:
# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["P2brain"]               = None#gamePad_policy
diambraGymKwargs["continue_game"]         = 0.0
diambraGymKwargs["show_final"]            = False
diambraGymKwargs["gamePads"]              = [None, diambraGymKwargs["P2brain"]]
diambraGymKwargs["actionSpace"]           = ["discrete", "multiDiscrete"]
diambraGymKwargs["attackButCombinations"] = [True, False]
diambraGymKwargs["actBufLen"]             = 12

In [5]:
envId = "Test" # This ID must be unique for every instance of the environment when using diambraGym class
env = diambraGym(envId, diambraEnvKwargs, **diambraGymKwargs)

Env_id = Test
Continue value = 0.0
Action Spaces = ['discrete', 'multiDiscrete']
Use attack buttons combinations = [True, False]
diambraEnv library successfully loaded
Pipe: /tmp/DIAMBRA/writeToDiambra-Test.pipe
Pipe: /tmp/DIAMBRA/readFromDiambra-Test.pipe
Using Discrete action space


In [6]:
# Printing action spaces
for idx in range(2):
    
    if diambraEnvKwargs["player"] != "P1P2" and idx == 1:
        continue
        
    print("Action space P{} = ".format(idx+1), env.action_spaces[idx])
    print("Action space type P{} = ".format(idx+1), env.action_spaces[idx].dtype)
    if diambraGymKwargs["actionSpace"][idx] == "multiDiscrete":
        print("Action space n = ", env.action_spaces[idx].nvec)
    else:
        print("Action space n = ", env.action_spaces[idx].n)

Action space P1 =  Discrete(16)
Action space type P1 =  int64
Action space n =  16


In [7]:
def showFrame(observation, waitKey = 1):
    obs = np.array(observation).astype(np.float32)/255

    cv2.imshow("image", obs[:, :, ::-1]) #bgr 2 rgb
    cv2.waitKey(waitKey)

In [8]:
actionsPrintDict = env.print_actions_dict
charNames = env.charNames

observation = env.reset()

In [ ]:
#showFrame(observation, 0) # Press space bar with image window active to complete cell execution

In [ ]:
tic = time.time()
fpsVal = []

while True:

    toc = time.time()
    fps = 1/(toc - tic)
    tic = toc
    #print("FPS = {}".format(fps))
    fpsVal.append(fps)    
    
    actions = [None, None]
    for idx in range(2):
        
        actions[idx] = env.action_spaces[idx].sample()
        
        if diambraGymKwargs["actionSpace"][idx] == "discrete":
            move_action, att_action = env.discreteToMultiDiscreteAction(actions[idx])
        else:
            move_action, att_action = actions[idx][0], actions[idx][1]

        if diambraEnvKwargs["player"] != "P1P2" and idx == 1:
            continue
            
        print("(P{}) {} {}".format(idx+1, actionsPrintDict[0][move_action],       
                                          actionsPrintDict[1][att_action])) 
        
    if diambraEnvKwargs["player"] != "P1P2" and diambraGymKwargs["actionSpace"][0] == "discrete":
        actions = actions[0]
    else:
        actions = np.append(actions[0], actions[1])    

    observation, reward, done, info = env.step(actions)
    
    print("Actions = ", actions)
    print("Reward        =", reward)
    print("Fighting      = ", info["fighting"])
    print("OwnCharacter  = ", charNames[info["ownCharacter"]])
    if diambraEnvKwargs["player"] == "P1P2":
        print("OppCharacter  = ", charNames[info["oppCharacter"]])
    print("OwnHealth     = ", info["ownHealth"])
    print("OppHealth     = ", info["oppHealth"])
    print("OwnPosition   = ", info["ownPosition"])
    print("OppPosition   = ", info["oppPosition"])
    print("OwnWin        = ", info["ownWins"])
    print("OppWin        = ", info["oppWins"])
    print("Stage         = ", info["stage"])
    print("OwnActionsBuf = ", info["actionsBufP1"])
    if diambraEnvKwargs["player"] == "P1P2":
        print("OppActionsBuf = ", info["actionsBufP2"])
    print("Round done    = ", info["round_done"])
    print("Stage done    = ", info["stage_done"])
    print("Game done     = ", info["game_done"])
    print("Episode done  = ", info["ep_done"])
    print("Done          = ", done)
    showFrame(observation,0)
    
    #if info["stage_done"] or done:
    #    break
    
    if done:
        print("Resetting Env")
        observation = env.reset()
        
env.close()

(P1)   Punch + Kick
Actions =  14
Reward        = 0
Fighting      =  1
OwnCharacter  =  Tina
OwnHealth     =  [208, 0]
OppHealth     =  [208, 0]
OwnPosition   =  0
OppPosition   =  1
OwnWin        =  0
OppWin        =  0
Stage         =  1
OwnActionsBuf =  [deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], maxlen=12), deque([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6], maxlen=12)]
Round done    =  False
Stage done    =  False
Game done     =  False
Episode done  =  False
Done          =  False


In [ ]:
print("Number of steps = ", len(fpsVal))
print("Average FPS = ", np.mean(fpsVal))
print("Std FPS = ", np.std(fpsVal))